In [1]:
# -*- coding: utf-8 -*-

# The program will consist of three steps:
#	(1) detect edges using the Sobel’s operator,
#	(2) detect straight line segments using the Hough Transform, and
#	(3) detect parallelograms from the straight-line segments detected in step (2).
# In step (1), compute edge magnitude using the formula below and
# then normalize the magnitude values to lie within the range [0,255].
# Next, manually choose a threshold value to produce a binary edge map.

import numpy as np
import matplotlib
from matplotlib import pylab as plt
import math
import itertools as it
from itertools import product
# import cv2
# from skimage.feature import peak_local_max
from photutils_detection_core import find_peaks
# import sys


row, col = 756, 1008  # Size of TestImage 1 and 2
# row, col = 413, 550 # Size of TestImage 3
filename = "TestImage1.raw"
T = 25  # Threshold in the normalized gradient magnitue
Canny_Edge_Detector_threshold = 10
local_maxima_window_size = 3  # neighborhood_size
# the de-Houghed image (using a relative threshold of 40%)
relative_threshold_ratio = 0.4
distance_threshold = 8  # Threshold distance to determin if a point on a line

# the least points on line to be considered to be a valid line
points_on_line_threshold = 20


In [2]:
# convert them into grayscale images by using the formula
# luminance = 0.30R + 0.59G + 0.11B,
# where R, G, and B, are the red, green, and blue components.


def cvt2grayscale(img):
    grayImage = []
    for i in range(0, img.size // 3):
        luminance = int(0.3 * img[3 * i] + 0.59 *
                        img[3 * i + 1] + 0.11 * img[3 * i + 2])
        grayImage.append(luminance)

    return np.array(grayImage)

# Gausssion smoothing: https://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm


def smooth_image_with_Gaussian_filter(img):
    kernel = (0.006, 0.061, 0.242, 0.383, 0.242, 0.061, 0.006)
    kernel_size = len(kernel)
    border_offset = (kernel_size - 1) // 2

    img_copy = np.copy(img)
    for i in range(0, row):
        # Keep border values as they are
        for j in range(border_offset, col - border_offset):
            img_copy_ij = 0
            for k in range((-1) * border_offset, border_offset + 1):
                img_copy_ij += img[i][j + k] * kernel[border_offset + k]
            img_copy[i][j] = img_copy_ij

    img_copy_copy = np.copy(img_copy)
    # Keep border values as they are
    for i in range(border_offset, row - border_offset):
        for j in range(0, col):
            img_copy_copy_ij = 0
            for k in range((-1) * border_offset, border_offset + 1):
                img_copy_copy_ij += img_copy[i +
                                             k][j] * kernel[border_offset + k]
            img_copy_copy[i][j] = img_copy_copy_ij

    return img_copy_copy



In [3]:
# Read Image
filepath = '../Parallelograms-Detection(Original Code)/TestImage1.raw'
testImage = np.fromfile(filepath, dtype='uint8', sep="")

# Convert to grayscale image
grayImage = cvt2grayscale(testImage).reshape([row, col])
print("Step 1: Convert image to grayscale.")
# print grayImage.shape

# Smooth_image_with_Gaussian_filter
grayImage_smoothed = smooth_image_with_Gaussian_filter(grayImage)


# Display Image
# plt.imshow(grayImage_smoothed, cmap='gray')
# plt.show()
# plt.savefig("grayImage_smoothed_with_Gaussian_filter.png")
# plt.close()


Step 1: Convert image to grayscale.


In [4]:
from numba import jit, float64, int64

In [7]:
# Compute gradient magnitude and gradient angle
gradient_magnitude = np.zeros((row, col), dtype='uint8')
gradient_angle = np.zeros((row, col), dtype='uint8')
quantize_angle_of_the_gradient = np.zeros((row, col), dtype='uint8')

@jit()
def quantize_angle_of_the_gradient_to_four_sectors_Final(angle):
    # Double check the parameter
    if (angle < 0 or angle > 360):
        print("Warning: invalid parameter in quantize_angle_of_the_gradient_to_four_sectors(angle).")
        return 4
    if (angle <= 22.5 or
            (angle >= 157.5 and angle <= 202.5) or
            angle >= 337.5):
        return 0
    if ((angle > 22.5 and angle < 67.5) or
            (angle > 202.5 and angle < 247.5)):
        return 1
    if ((angle >= 67.5 and angle <= 112.5) or
            (angle >= 247.5 and angle <= 292.5)):
        return 2
    if ((angle > 112.5 and angle < 157.5) or
            (angle > 292.5 and angle < 337.5)):
        return 3

@jit
def compute_gradient_magnitude_and_gradient_angle_Final(image_smoothed):
    indices = product(range(1,row),range(1,col))
    for i in indices:
        Gx = (image_smoothed[i] + image_smoothed[i[0]-1][i[1]]
                  - image_smoothed[i[0]][i[1]-1] - image_smoothed[i[0]-1][i[1]-1])
        Gy = (image_smoothed[i[0]-1][i[1]-1] + image_smoothed[i[0]-1][i[1]]
                  - image_smoothed[i[0]][i[1]-1] - image_smoothed[i])
        gradient_magnitude[i] = math.sqrt(Gx * Gx + Gy * Gy)
        if Gx == 0:
            gradient_angle[i] = 90 if Gy > 0 else 270
        else:
            gradient_angle[i] = math.degrees(math.atan2(Gy, Gx))

        quantize_angle_of_the_gradient[i] = quantize_angle_of_the_gradient_to_four_sectors_Final(gradient_angle[i])

In [9]:
%timeit -n 2 compute_gradient_magnitude_and_gradient_angle_Final(grayImage_smoothed)

3.72 s ± 327 ms per loop (mean ± std. dev. of 7 runs, 2 loops each)


In [10]:
imgMag = gradient_magnitude
theta_step_size = 3
p_step_size = 1
theta_MAX_VALUE = 360
p_MAX_VALUE = int(math.sqrt(row * row + col * col))

accumulator_array = np.zeros(
        (theta_MAX_VALUE // theta_step_size + 1, p_MAX_VALUE // p_step_size + 1), dtype='uint8')
    # Compute the accumulator array
imgMag_row, imgMag_col = imgMag.shape
    
indices = product(range(0,imgMag_row),range(0,imgMag_col))
for i in indices:
    if(imgMag[i] > 0):
        # p = x*cos(theta) + y*sin(theta)
        theta = 0
        for theta in range(0,360,3):
            theta_radians = math.radians(theta + theta_step_size / 2.0)
            p_estimate = i[0] * math.cos(theta_radians) + \
                        i[1] * math.sin(theta_radians)
            # Update the accumulator array
            accu_x = theta // theta_step_size
            accu_y = int(p_estimate / p_step_size)
            accumulator_array[accu_x][accu_y] += 1
               
max_accumulator = np.amax(accumulator_array)
print(max_accumulator)
print("Step 3: Hough Transform applied.")

255
Step 3: Hough Transform applied.


In [7]:
import math

In [9]:
360//7

51